In [2]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import face_recognition
import cv2 as cv
import os
from PIL import Image

In [ ]:
x_path = "testing/X"
y_path = "testing/Y"

In [ ]:
def read_data(x_path, y_path):
    X = []
    Y = []
    
    with os.scandir(x_path) as entries:
        for entry in entries:
            X.append(keras.preprocessing.image.img_to_array(
                keras.preprocessing.image.load_img(
                    os.path.join(x_path, entry.name)
            ))/255.0)
            Y.append(keras.preprocessing.image.img_to_array(
                keras.preprocessing.image.load_img(
                    os.path.join(y_path, entry.name)
            ))/255.0)
        
    return np.array(X), np.array(Y)

In [ ]:
picture_names = os.listdir(os.path.join(os.getcwd(), "testing", "X"))

In [ ]:
X, Y = read_data(x_path, y_path)
X_test, Y_test = X, Y

In [ ]:
print(X_test.shape)
print(Y_test.shape)

In [ ]:
def face_embedding_compare(x, y):
    image_x = cv.imread(x)
    rgb_x = cv.cvtColor(image_x, cv.COLOR_BGR2RGB)
    image_y = cv.imread(y)
    rgb_y = cv.cvtColor(image_y, cv.COLOR_BGR2RGB)
    boxes_x = face_recognition.face_locations(rgb_x, model="hog")
    boxes_y = face_recognition.face_locations(rgb_y, model="hog")
    if boxes_x == []:
        return "could not identify face"
    encoding_x = face_recognition.face_encodings(rgb_x, boxes_x)
    encoding_y = face_recognition.face_encodings(rgb_y, boxes_y)
    np_x = np.asarray(encoding_x)
    np_y = np.asarray(encoding_y)
    euc_dis = np.linalg.norm(np_x - np_y, ord=2)
    return f"{euc_dis}"

In [ ]:
model = keras.models.load_model("network/model.tf", compile=False)

In [ ]:
t = model.predict(X_test)

In [ ]:
images = len(picture_names)

for x in range(images):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,15))
    ax1.imshow(X_test[x])
    ax2.imshow(Y_test[x])
    ax3.imshow(t[x])
    
    name = f"{picture_names[x]}"
    locX = f"testing/X/{name}"
    locY = f"testing/Y/{name}"
    locfX = f"testing/fX/{name}"
    
    array = t[x] * 255
    array = array.astype("uint8")
    im = Image.fromarray(array)
    im.save(locfX)
    
    x = face_embedding_compare(locX, locY)
    fx = face_embedding_compare(locfX, locY)

    ax2.set(title=name)
    ax1.set(xlabel=x)
    ax2.set(xlabel='0')
    ax3.set(xlabel=fx)

    plt.show()